###  importing relevant libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from collections import Counter
from tabulate import tabulate
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline
import warnings
import sklearn
import time
import os
warnings.filterwarnings("ignore")
from sklearn.metrics import confusion_matrix


### Discovering Labels

In [3]:
def target_name(name):
    df = pd.read_csv(name,usecols=["Label"])
    df["Label"]=df["Label"].replace(renamer)
    df=df.dropna()
    target_names=sorted(list(df["Label"].unique()))
    return target_names

In [4]:
def folder(f_name): #this function creates a folder.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("Tthe folder could not be created!")

In [5]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [6]:
renamer={}

In [7]:
altime=0
#def most_frequent(List): 
#    return max(set(List), key = List.count) 


 
def most_frequent(List):
    occurence_count = Counter(List)
    occurence_count={k: v for k, v in sorted(occurence_count.items(), key=lambda item: item[1],reverse=True)}
    big=list(occurence_count.values())
    big=big.count(big[0])
    return list(occurence_count.keys())[np.random.randint(big)]


def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))
def create_exception(df): 
    exception_list=[]
    dominant_mac=[]
    for i in df['aggregated'].unique():
        k=df[df['aggregated']==i]
        for ii in ['MAC']:
            hist = {}
            for x in k[ii].values:
                hist[x] = hist.get(x, 0) + 1
            hist=dict(sorted(hist.items(), key=lambda item: item[1],reverse=True))
            temp=next(iter(hist))
            if temp not in dominant_mac:
                dominant_mac.append(temp)
            else:
                exception_list.append(temp)
    return exception_list




def merged(m_test,predict,step,mixed):
    second=time.time()
    mac_test=[]
    for q in m_test.index:
        mac_test.append(m_test[q])

    d_list=sorted(list(m_test.unique()))
    devices={}
    for q in d_list:
        devices[q]=[]    


    new_y=[0]*len(m_test)

    for q,qq in enumerate (mac_test):
        devices[qq].append(q)
    for q in devices:
        a = [devices[q][j:j + step] for j in range(0, len(devices[q]), step)]  
        for qq in a:
            step_list=[]
            for qqq in qq:
                step_list.append(predict[qqq])
            add=most_frequent(list(step_list))
            for qqq in qq:
                new_y[qqq]=add
    results=pd.DataFrame(m_test)
    results["aggregated"]=new_y
    results["normal"]=predict
    
    #MIXED METHOD
    if mixed:
        exception=create_exception(results)
        for q in exception:
            results.loc[results.MAC == q, 'aggregated'] = results['normal']

    return results["aggregated"].values,time.time()-second

## Calculation of evaluations

In [8]:
def score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ml):
    rc=sklearn.metrics.recall_score(y_test, predict,average= "macro")
    pr=sklearn.metrics.precision_score(y_test, predict,average= "macro")
    f_1=sklearn.metrics.f1_score(y_test, predict,average= "macro")     
    accuracy=sklearn.metrics.accuracy_score(y_test, predict)
    accuracy_b=sklearn.metrics.balanced_accuracy_score( y_test,predict)
    kappa=sklearn.metrics.cohen_kappa_score(y_test, predict,labels=None, weights=None, sample_weight=None)
    try:
        roc=sklearn.metrics.roc_auc_score(y_test, predict)
    except:roc=0
    report = sklearn.metrics.classification_report(y_test, predict, target_names=target_names,output_dict=True)
    cr = pd.DataFrame(report).transpose()
    line=[dname,i,cv,ml,accuracy,accuracy_b,pr,rc,f_1,kappa,roc,train_time,test_time,altime]

    if class_based_results.empty:
        class_based_results =cr
    else:
        class_based_results = class_based_results.add(cr, fill_value=0)
    return line,class_based_results

In [9]:


def Diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif
 
# Driver Code
#li1 = [10, 15, 20, 25, 30, 35, 40]
#li2 = [25, 40, 35]


In [10]:
def ML(loop1,loop2,output_csv1,cols,step,mixed,dname):

    
    for ii in ml_list:

        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        output_csv=output_csv1.replace("ML",ii)
        repetition=5
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]
        for i in range(repetition):



            #TRAIN
            df = pd.read_csv(loop1,usecols=cols)
            df=df.dropna()
            df["Label"]=df["Label"].replace(renamer)
            X_train =df[df.columns[0:-1]]
            X_train=np.array(X_train)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_train=df[df.columns[-1]].cat.codes  
            
            

            #TEST
            df = pd.read_csv(loop2,usecols=cols)
            df=df.dropna()
            df["Label"]=df["Label"].replace(renamer)
            df = shuffle(df, random_state=42)

            X_test =df[df.columns[0:-1]]
            X_test=np.array(X_test)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_test=df[df.columns[-1]].cat.codes


            results_y=[]
            cv+=1
            results_y.append(y_test)


            #machine learning algorithm is applied in this section
            clf = ml_list[ii]#choose algorithm from ml_list dictionary
            second=time.time()
            clf.fit(X_train, y_train)
            train_time=(float((time.time()-second)) )
            second=time.time()
            predict =clf.predict(X_test)
            test_time=(float((time.time()-second)) )
            if step==1:
                altime=0
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
            else:
                predict,altime=merged(m_test,predict,step,mixed)
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                
                
            lines.append(line)
            df_cm = pd.DataFrame(confusion_matrix(y_test, predict))
            if cm.empty:
                cm =df_cm
            else:
                cm = cm.add(df_cm, fill_value=0)
            
        class_based_results=class_based_results/(repetition)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()
  
        
        print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if True:
            cm=cm//repetition
            cm.to_csv(output_csv.replace(".csv","CM.csv"))
            #graph_name=output_csv+"_confusion matrix.pdf"   
            #plt.figure(figsize = (40,28))
            #sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            #plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            #plt.show()
            #print(cm)
            #print("\n\n\n")             


            #plt.clf()

            
            
            
            
            



# Machine learning applications 

# Aalto Dataset

In [11]:
feature= [  'dstport',
  'dstport_class',
  'http.chat',
  'http.notification',
  'http.request.method',
  'ip.flags.df',
  'ip.len',
  'ip.proto',
  'ip.ttl',
  'srcport',
  'srcport_class',
  'tcp.ack',
  'tcp.analysis.ack_rtt',
  'tcp.analysis.bytes_in_flight',
  'tcp.analysis.initial_rtt',
  'tcp.analysis.push_bytes_sent',
  'tcp.completeness',
  'tcp.dstport',
  'tcp.flags',
  'tcp.flags.push',
  'tcp.flags.str',
  'tcp.flags.syn',
  'tcp.hdr_len',
  'tcp.len',
  'tcp.nxtseq',
  'tcp.srcport',
  'tcp.stream',
  'tcp.time_delta',
  'tcp.time_relative',
  'tcp.window_size',
  'tcp.window_size_scalefactor',
  'tcp.window_size_value',
  'tls.record.length',
  'udp.checksum.status',
  'udp.dstport',
  'udp.srcport',
  'udp.time_delta',
  'udp.time_relative',"Label"]

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost

In [13]:

ml_list={"DT":DecisionTreeClassifier(criterion="entropy",max_depth=14,max_features=21,min_samples_split=5 ),        
"RF":RandomForestClassifier(bootstrap=False,criterion="entropy",max_depth=17,max_features=3,min_samples_split=5,n_estimators=71),         
"XGB":xgboost.XGBClassifier(n_estimators=900,min_child_weight=1,max_depth=3,learning_rate=0.15),
"KNN":KNeighborsClassifier(algorithm="kd_tree",leaf_size=39,n_neighbors=5,weights="distance"),
"NB":GaussianNB(var_smoothing=1.51991e-06), 

"LR":LogisticRegression(C=0.0809264 ,penalty="l2",solver="newton-cg" ),"SVM":SVC(gamma=0.001,C=1)}

#ml_list={"DT":DecisionTreeClassifier(criterion="entropy",max_depth=14,max_features=21,min_samples_split=5 )}


In [14]:
find_the_way("./","clean")

[]

In [15]:
file_list={"DD11":['./small/AD-S1.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
"DD11T":['./small/DI-S1.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
"DD12":['./small/AD-S1.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
"DD12T":['./small/DI-S2.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
"DD22":['./small/AD-S2.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
"DD22T":['./small/DI-S2.csv','./small/AD-S2.csv','./small/AD-S1.csv'],
"DD21":['./small/AD-S2.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
"DD21T":['./small/DI-S1.csv','./small/AD-S2.csv','./small/AD-S1.csv']}
#file_list={"DD11":['./small/AD-S1.csv','./small/DI-S1.csv','./small/DI-S2.csv']}

In [32]:
say=0
dataset="./results/other/1/"
folder(dataset)

for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=1
        sayac=1
        output_csv=f"{dataset}/ML@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}.csv"
        dname=(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}")
        print(dname)
        target_names=target_name(test)
        ML(train,test,output_csv,feature,step,mixed,dname)   



AD-S1-DI-S1
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S1-DI-S1    0     1  DT    0.724    0.751   0.739  0.751  0.706  0.71       0    0.888     0.029          0
 1  AD-S1-DI-S1    1     2  DT    0.652    0.681   0.736  0.681  0.641  0.634      0    0.921     0.023          0
 2  AD-S1-DI-S1    2     3  DT    0.671    0.708   0.723  0.708  0.662  0.654      0    0.77      0.028          0
 3  AD-S1-DI-S1    3     4  DT    0.72     0.756   0.754  0.756  0.695  0.706      0    0.838     0.03           0
 4  AD-S1-DI-S1    4     5  DT    0.702    0.735   0.77   0.735  0.697  0.687      0    1.049     0.04           0

                                  precision     recall    f1-score       support
------------------------------  -----------  ---------  ----------  ------------
Amazon Echo        

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S1-DI-S1    0     1  KNN   0.632    0.643   0.669  0.643  0.607  0.612      0    1.69     12.329          0
 1  AD-S1-DI-S1    1     2  KNN   0.632    0.643   0.669  0.643  0.607  0.612      0    2.109    12.509          0
 2  AD-S1-DI-S1    2     3  KNN   0.632    0.643   0.669  0.643  0.607  0.612      0    2.09     13.652          0
 3  AD-S1-DI-S1    3     4  KNN   0.632    0.643   0.669  0.643  0.607  0.612      0    1.901    12.071          0
 4  AD-S1-DI-S1    4     5  KNN   0.632    0.643   0.669  0.643  0.607  0.612      0    1.112    11.017          0

                                  precision     recall    f1-score       support
------------------------------  -----------  ---------  ----------  ------------
Amazon Echo                     

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec    F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  ----  -----  -----  -------  --------  ---------
 0  AD-S1-DI-S1    0     1  SVM   0.098    0.092   0.467  0.092  0.08  0.049      0  1987.48   1006.83          0
 1  AD-S1-DI-S1    1     2  SVM   0.098    0.092   0.467  0.092  0.08  0.049      0  2238.22   1003.6           0
 2  AD-S1-DI-S1    2     3  SVM   0.098    0.092   0.467  0.092  0.08  0.049      0  1969.58   1005.38          0
 3  AD-S1-DI-S1    3     4  SVM   0.098    0.092   0.467  0.092  0.08  0.049      0  1960.13   1011.14          0
 4  AD-S1-DI-S1    4     5  SVM   0.098    0.092   0.467  0.092  0.08  0.049      0  1957.44   1006.99          0

                                  precision       recall     f1-score        support
------------------------------  -----------  -----------  -----------  -------------
Amazon Echo                    

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  DI-S1-AD-S1    0     1  XGB   0.795    0.814   0.827  0.814  0.811  0.784      0  143.186     6.929          0
 1  DI-S1-AD-S1    1     2  XGB   0.795    0.814   0.827  0.814  0.811  0.784      0  143.011     6.937          0
 2  DI-S1-AD-S1    2     3  XGB   0.795    0.814   0.827  0.814  0.811  0.784      0  143.204     7.021          0
 3  DI-S1-AD-S1    3     4  XGB   0.795    0.814   0.827  0.814  0.811  0.784      0  142.405     7.002          0
 4  DI-S1-AD-S1    4     5  XGB   0.795    0.814   0.827  0.814  0.811  0.784      0  142.004     7              0

                                  precision    recall    f1-score       support
------------------------------  -----------  --------  ----------  ------------
Amazon Echo                       

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  DI-S1-AD-S1    0     1  LR    0.609    0.609   0.655  0.609  0.593  0.587      0  400.456     0.031          0
 1  DI-S1-AD-S1    1     2  LR    0.609    0.609   0.655  0.609  0.593  0.587      0  404.955     0.03           0
 2  DI-S1-AD-S1    2     3  LR    0.609    0.609   0.655  0.609  0.593  0.587      0  404.887     0.028          0
 3  DI-S1-AD-S1    3     4  LR    0.609    0.609   0.655  0.609  0.593  0.587      0  400.695     0.03           0
 4  DI-S1-AD-S1    4     5  LR    0.609    0.609   0.655  0.609  0.593  0.587      0  404.067     0.031          0

                                  precision    recall    f1-score       support
------------------------------  -----------  --------  ----------  ------------
Amazon Echo                       

...

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  DI-S2-AD-S2    0     1  LR    0.635    0.651    0.69  0.651  0.647  0.615      0  347.422     0.018          0
 1  DI-S2-AD-S2    1     2  LR    0.635    0.651    0.69  0.651  0.647  0.615      0  346.939     0.028          0
 2  DI-S2-AD-S2    2     3  LR    0.635    0.651    0.69  0.651  0.647  0.615      0  348.928     0.031          0
 3  DI-S2-AD-S2    3     4  LR    0.635    0.651    0.69  0.651  0.647  0.615      0  347.033     0.022          0
 4  DI-S2-AD-S2    4     5  LR    0.635    0.651    0.69  0.651  0.647  0.615      0  346.064     0.031          0

                                  precision     recall    f1-score      support
------------------------------  -----------  ---------  ----------  -----------
Amazon Echo                       

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S2-DI-S1    0     1  RF    0.809    0.822   0.845  0.822  0.785  0.799      0    7.012     1.253          0
 1  AD-S2-DI-S1    1     2  RF    0.811    0.823   0.844  0.823  0.785  0.801      0    6.919     1.207          0
 2  AD-S2-DI-S1    2     3  RF    0.804    0.818   0.838  0.818  0.779  0.794      0    6.947     1.197          0
 3  AD-S2-DI-S1    3     4  RF    0.8      0.816   0.835  0.816  0.775  0.789      0    6.886     1.217          0
 4  AD-S2-DI-S1    4     5  RF    0.799    0.814   0.836  0.814  0.775  0.789      0    7.428     1.223          0

                                  precision     recall    f1-score       support
------------------------------  -----------  ---------  ----------  ------------
Amazon Echo                     

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S2-DI-S1    0     1  NB    0.433    0.422    0.36  0.422  0.372  0.403      0    0.061     0.516          0
 1  AD-S2-DI-S1    1     2  NB    0.433    0.422    0.36  0.422  0.372  0.403      0    0.063     0.516          0
 2  AD-S2-DI-S1    2     3  NB    0.433    0.422    0.36  0.422  0.372  0.403      0    0.053     0.536          0
 3  AD-S2-DI-S1    3     4  NB    0.433    0.422    0.36  0.422  0.372  0.403      0    0.065     0.536          0
 4  AD-S2-DI-S1    4     5  NB    0.433    0.422    0.36  0.422  0.372  0.403      0    0.069     0.537          0

                                  precision     recall    f1-score       support
------------------------------  -----------  ---------  ----------  ------------
Amazon Echo                     

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  DI-S1-AD-S2    0     1  DT    0.656    0.682   0.723  0.682  0.678  0.636      0    0.519     0.02           0
 1  DI-S1-AD-S2    1     2  DT    0.613    0.638   0.637  0.638  0.599  0.592      0    0.581     0.024          0
 2  DI-S1-AD-S2    2     3  DT    0.717    0.742   0.739  0.742  0.706  0.702      0    0.527     0.02           0
 3  DI-S1-AD-S2    3     4  DT    0.737    0.744   0.722  0.744  0.714  0.723      0    0.485     0.02           0
 4  DI-S1-AD-S2    4     5  DT    0.707    0.73    0.791  0.73   0.735  0.691      0    0.539     0.018          0

                                  precision    recall    f1-score       support
------------------------------  -----------  --------  ----------  ------------
Amazon Echo                       

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  DI-S1-AD-S2    0     1  NB    0.447     0.47   0.562   0.47  0.424  0.418      0    0.071     0.495          0
 1  DI-S1-AD-S2    1     2  NB    0.447     0.47   0.562   0.47  0.424  0.418      0    0.071     0.486          0
 2  DI-S1-AD-S2    2     3  NB    0.447     0.47   0.562   0.47  0.424  0.418      0    0.071     0.495          0
 3  DI-S1-AD-S2    3     4  NB    0.447     0.47   0.562   0.47  0.424  0.418      0    0.069     0.497          0
 4  DI-S1-AD-S2    4     5  NB    0.447     0.47   0.562   0.47  0.424  0.418      0    0.071     0.516          0

                                  precision      recall    f1-score      support
------------------------------  -----------  ----------  ----------  -----------
Amazon Echo                     

In [33]:
#!shutdown /s /t 360

# MLP

In [16]:
import sklearn
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
def create_model(params):
    model = Sequential()

    # Giriş katmanı
    model.add(Dense(units=params['units_input'], activation='relu', input_shape=(input_shape,)))

    # Gizli katmanlar
    for i in range(params['num_layers']):
        model.add(Dense(units=params[f'units_{i}'], activation='relu'))
        model.add(Dropout(params[f'dropout_{i}']))

    # Çıkış katmanı
    model.add(Dense(num_classes, activation='softmax'))

    # Optimizasyon ve kayıp fonksiyonunun belirlenmesi
    optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

best_params = {'units_input': 128, 'num_layers': 3, 'units_0': 32, 'dropout_0': 0.0, 
               'learning_rate': 0.01, 'units_1': 64, 'dropout_1': 0.0, 'units_2': 96, 'dropout_2': 0.0}
say=0
dataset="./results/other/1/"
folder(dataset)

for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=1
        sayac=1
        output_csv=f"{dataset}/MLP@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}.csv"
        print(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}")
        target_names=target_name(test)


        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
    
        repetition=5
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]


    
        for i in range(repetition):

            df = pd.read_csv(train,usecols=feature)
            df=df.dropna()
            df["Label"]=df["Label"].replace(renamer)
            X_train =df[df.columns[0:-1]]
            X_train=np.array(X_train)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_train=df[df.columns[-1]].cat.codes  
            
            

            #TEST
            df = pd.read_csv(test,usecols=feature)
            df=df.dropna()
            df["Label"]=df["Label"].replace(feature)
            df = shuffle(df, random_state=42)

            X_test =df[df.columns[0:-1]]
            X_test=np.array(X_test)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_test=df[df.columns[-1]].cat.codes
            
            y_train = to_categorical(y_train)
            #y_test = to_categorical(y_test)
            min_max_scaler = preprocessing.MinMaxScaler()
            X_train = min_max_scaler.fit_transform(X_train)
            X_test = min_max_scaler.fit_transform(X_test)
            
            input_shape=X_train.shape[1]
            num_classes=len(y_test.unique())
            # Modelin oluşturulması

            results_y=[]
            cv+=1
            results_y.append(y_test)

            model = create_model(best_params)
            #model.summary()
            second=time.time()
            model = create_model(best_params)
            model.fit(X_train, y_train, epochs=10, batch_size=32,validation_split=0.3,verbose= 0)#validation_data=(X_test, y_test)) #validation_split=0.3)
            train_time=(float((time.time()-second)) )
            second=time.time()
            
            predict =model.predict(X_test)
            predict = np.argmax(predict, axis=1)
            test_time=(float((time.time()-second)) )
            
            
            altime=0
            dname=f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}"
            line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,"MLP")
            
                
                
              
            lines.append(line)
            df_cm = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test, predict))
            if cm.empty:
                cm =df_cm
            else:
                cm = cm.add(df_cm, fill_value=0)
                        
        class_based_results=class_based_results/(repetition)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv,index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()
  
        
        print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if True:
            cm=cm//repetition
            cm.to_csv(output_csv.replace(".csv","CM.csv"))
            #graph_name=output_csv+"_confusion matrix.pdf"   
            #plt.figure(figsize = (40,28))
            #sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            #plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            #plt.show()
            #print(cm)
            #print("\n\n\n")             


            #plt.clf()
  


AD-S1-DI-S1
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S1-DI-S1    0     1  MLP   0.493    0.512   0.594  0.512  0.482  0.466      0   37.92      3.07           0
 1  AD-S1-DI-S1    1     2  MLP   0.593    0.614   0.623  0.614  0.582  0.571      0   39.246     2.657          0
 2  AD-S1-DI-S1    2     3  MLP   0.49     0.509   0.592  0.509  0.49   0.462      0   37.263     2.814          0
 3  AD-S1-DI-S1    3     4  MLP   0.53     0.554   0.609  0.554  0.514  0.505      0   29.061     1.904          0
 4  AD-S1-DI-S1    4     5  MLP   0.487    0.499   0.595  0.499  0.477  0.459      0   27.531     2.015          0

                                  precision     recall    f1-score       support
------------------------------  -----------  ---------  ----------  ------------
Amazon Echo        

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  DI-S2-AD-S1    0     1  MLP   0.418    0.396   0.463  0.396  0.378  0.383      0   27.976     1.887          0
 1  DI-S2-AD-S1    1     2  MLP   0.343    0.318   0.357  0.318  0.289  0.303      0   27.515     1.935          0
 2  DI-S2-AD-S1    2     3  MLP   0.35     0.306   0.345  0.306  0.282  0.311      0   29.253     1.836          0
 3  DI-S2-AD-S1    3     4  MLP   0.358    0.361   0.396  0.361  0.337  0.322      0   27.648     1.982          0
 4  DI-S2-AD-S1    4     5  MLP   0.333    0.303   0.363  0.303  0.266  0.292      0   28.446     2.008          0

                                  precision      recall    f1-score       support
------------------------------  -----------  ----------  ----------  ------------
Amazon Echo                   

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S2-DI-S1    0     1  MLP   0.486    0.504   0.475  0.504  0.457  0.459      0   26.242     1.888          0
 1  AD-S2-DI-S1    1     2  MLP   0.514    0.536   0.558  0.536  0.482  0.488      0   26.404     2.089          0
 2  AD-S2-DI-S1    2     3  MLP   0.505    0.524   0.477  0.524  0.468  0.479      0   26.6       1.988          0
 3  AD-S2-DI-S1    3     4  MLP   0.505    0.522   0.501  0.522  0.472  0.478      0   27.831     1.986          0
 4  AD-S2-DI-S1    4     5  MLP   0.452    0.488   0.452  0.488  0.421  0.425      0   26.863     1.987          0

                                  precision     recall    f1-score       support
------------------------------  -----------  ---------  ----------  ------------
Amazon Echo                     

# CNN

In [17]:
import sklearn
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing

def create_model(params):
    model = Sequential()

    # Convolutional layer
    model.add(Conv1D(filters=params['conv1d_filters'], kernel_size=params['conv1d_kernel_size'], activation='relu', input_shape=((X_train.shape[1], X_train.shape[2]))))

    # Pooling layer
    model.add(MaxPooling1D(pool_size=params['max_pooling1d_pool_size']))

    # Flatten layer
    model.add(Flatten())

    # Dense layers
    model.add(Dense(units=params['dense_units_1'], activation='relu'))
    model.add(Dropout(params['dropout_rate_1']))
    model.add(Dense(units=params['dense_units_2'], activation='relu'))
    model.add(Dropout(params['dropout_rate_2']))
    model.add(Dense(units=params['dense_units_3'], activation='softmax'))

    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

best_params = {
    "conv1d_filters": 32,
    "conv1d_kernel_size": 3,
    "max_pooling1d_pool_size": 2,
    "dense_units_1": 32,
    "dropout_rate_1": 0.0,  # No dropout after the first dense layer
    "dense_units_2": 96,
    "dropout_rate_2": 0.0,  # No dropout after the second dense layer
    "dense_units_3": 21,
    "learning_rate": 0.01
}

say = 0
dataset = "./results/other/1/"
folder(dataset)

for ii in file_list:
    print()
    say += 1
    train = file_list[ii][0]
    test = file_list[ii][1]
    mixed = False
    step = 1
    sayac = 1
    output_csv = f"{dataset}/CNN@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}.csv"
    print(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}")
    target_names = target_name(test)

    class_based_results = pd.DataFrame()
    cm = pd.DataFrame()
    cv = 0
    repetition = 5
    lines = [["Dataset", "T", "CV", "ML", "Acc", "b_Acc", "Prec", "Rec", "F1", "kap", "ROC", "tra-T", "test-T", "Al-Time"]]

    for i in range(repetition):
        df = pd.read_csv(train, usecols=feature)
        df = df.dropna()
        df["Label"] = df["Label"].replace(renamer)
        X_train = df[df.columns[0:-1]]
        X_train = np.array(X_train)
        df[df.columns[-1]] = df[df.columns[-1]].astype('category')
        y_train = df[df.columns[-1]].cat.codes  

        df = pd.read_csv(test, usecols=feature)
        df = df.dropna()
        df["Label"] = df["Label"].replace(feature)
        df = shuffle(df, random_state=42)

        X_test = df[df.columns[0:-1]]
        X_test = np.array(X_test)
        df[df.columns[-1]] = df[df.columns[-1]].astype('category')
        y_test = df[df.columns[-1]].cat.codes

        y_train = to_categorical(y_train)
        min_max_scaler = preprocessing.MinMaxScaler()
        X_train = min_max_scaler.fit_transform(X_train)
        X_test = min_max_scaler.fit_transform(X_test)
        # Veriyi CNN modeline uygun hale getirme
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
        
        input_shape = X_train.shape[1]
        num_classes = len(y_test.unique())

        results_y = []
        cv += 1
        results_y.append(y_test)

        model = create_model(best_params)
        second = time.time()
        model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.3, verbose=0)
        train_time = (float((time.time() - second)))
        second = time.time()

        predict = model.predict(X_test)
        predict = np.argmax(predict, axis=1)
        test_time = (float((time.time() - second)))

        altime = 0
        dname = f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}"
        line, class_based_results = score(altime, train_time, test_time, predict, y_test, class_based_results, i, cv, dname, "CNN")

        lines.append(line)
        df_cm = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test, predict))
        if cm.empty:
            cm = df_cm
        else:
            cm = cm.add(df_cm, fill_value=0)

    class_based_results = class_based_results / repetition
    results = pd.DataFrame(lines[1:], columns=lines[0])
    results.to_csv(output_csv, index=False)
    results = results.round(3)
    print(tabulate(results, headers=list(results.columns)))
    print()

    print(tabulate(class_based_results, headers=list(class_based_results.columns)))
    class_based_results.to_csv(output_csv.replace(".csv", "CB.csv"))
    if True:
        cm = cm // repetition
        cm.to_csv(output_csv.replace(".csv", "CM.csv"))



AD-S1-DI-S1
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S1-DI-S1    0     1  CNN   0.43     0.454   0.52   0.454  0.434  0.399      0   43.517     2.922          0
 1  AD-S1-DI-S1    1     2  CNN   0.354    0.349   0.459  0.349  0.319  0.319      0   48.704     2.768          0
 2  AD-S1-DI-S1    2     3  CNN   0.264    0.262   0.263  0.262  0.224  0.224      0   49.778     2.836          0
 3  AD-S1-DI-S1    3     4  CNN   0.434    0.441   0.518  0.441  0.42   0.403      0   47.789     3.221          0
 4  AD-S1-DI-S1    4     5  CNN   0.277    0.262   0.363  0.262  0.245  0.238      0   47.349     2.62           0

                                  precision     recall    f1-score       support
------------------------------  -----------  ---------  ----------  ------------
Amazon Echo        

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  DI-S2-AD-S1    0     1  CNN   0.386    0.353   0.473  0.353  0.332  0.349      0   48.267     2.804          0
 1  DI-S2-AD-S1    1     2  CNN   0.3      0.267   0.329  0.267  0.242  0.258      0   47.225     2.974          0
 2  DI-S2-AD-S1    2     3  CNN   0.465    0.458   0.572  0.458  0.441  0.436      0   47.159     2.921          0
 3  DI-S2-AD-S1    3     4  CNN   0.442    0.455   0.491  0.455  0.439  0.41       0   49.581     2.975          0
 4  DI-S2-AD-S1    4     5  CNN   0.398    0.382   0.495  0.382  0.375  0.363      0   50.31      3.239          0

                                  precision      recall    f1-score      support
------------------------------  -----------  ----------  ----------  -----------
Amazon Echo                     

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S2-DI-S1    0     1  CNN   0.442    0.447   0.473  0.447  0.427  0.413      0   47.309     3.032          0
 1  AD-S2-DI-S1    1     2  CNN   0.383    0.382   0.398  0.382  0.34   0.351      0   47.281     3.071          0
 2  AD-S2-DI-S1    2     3  CNN   0.447    0.461   0.492  0.461  0.431  0.418      0   45.155     3.043          0
 3  AD-S2-DI-S1    3     4  CNN   0.383    0.389   0.436  0.389  0.362  0.349      0   46.387     2.769          0
 4  AD-S2-DI-S1    4     5  CNN   0.443    0.455   0.49   0.455  0.431  0.414      0   44.969     3.407          0

                                  precision     recall    f1-score       support
------------------------------  -----------  ---------  ----------  ------------
Amazon Echo                     

# LSTM

In [19]:
import sklearn
import keras
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers 
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing



say = 0
dataset = "./results/other/1/"
folder(dataset)

for ii in file_list:
    print()
    say += 1
    train = file_list[ii][0]
    test = file_list[ii][1]
    mixed = False
    step = 1
    sayac = 1
    output_csv = f"{dataset}/LSTM@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}.csv"
    print(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}")
    target_names = target_name(test)

    class_based_results = pd.DataFrame()
    cm = pd.DataFrame()
    cv = 0
    repetition = 5
    lines = [["Dataset", "T", "CV", "ML", "Acc", "b_Acc", "Prec", "Rec", "F1", "kap", "ROC", "tra-T", "test-T", "Al-Time"]]

    for i in range(repetition):
        # Veri yükleme ve hazırlama
        df_train = pd.read_csv(file_list[ii][0], usecols=feature) 
        X_train = df_train.iloc[:, :-1]
        df_train['Label'] = df_train['Label'].astype('category')
        y_train = df_train['Label'].cat.codes  
        
        df_test = pd.read_csv(file_list[ii][1], usecols=feature) 
        X_test = df_test.iloc[:, :-1]
        df_test['Label'] = df_test['Label'].astype('category')
        y_test = df_test['Label'].cat.codes  
        
        min_max_scaler = preprocessing.MinMaxScaler()
        X_train = min_max_scaler.fit_transform(X_train)
        X_test = min_max_scaler.transform(X_test)
        
        # Veriyi LSTM modeline uygun hale getirme
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
        
        # Modelin oluşturulması
        model = keras.Sequential()
        model.add(layers.LSTM(units=32,
                              return_sequences=True,
                              input_shape=(X_train.shape[1], X_train.shape[2])))
        model.add(layers.Dropout(rate=0.1))
        
        model.add(layers.LSTM(units=64, return_sequences=True))
        model.add(layers.Dropout(rate=0.4))
        
        model.add(layers.LSTM(units=96, return_sequences=True))
        model.add(layers.Dropout(rate=0.3))
        
        model.add(layers.LSTM(units=96))
        model.add(layers.Dropout(rate=0.0))
        
        model.add(layers.Dense(21, activation='softmax'))  # 21 sınıf için
        
        model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01),
                      loss='sparse_categorical_crossentropy',
                      metrics=["accuracy"])
        
        second = time.time()
        model.fit(X_train, y_train, epochs=10,validation_split=0.3, verbose=1)
        results_y = []
        cv += 1
        results_y.append(y_test)
        train_time = (float((time.time() - second)))
        second = time.time()

        predict = model.predict(X_test)
        predict = np.argmax(predict, axis=1)
        test_time = (float((time.time() - second)))

        altime = 0
        dname = f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}"
        line, class_based_results = score(altime, train_time, test_time, predict, y_test, class_based_results, i, cv, dname, "LSTM")

        lines.append(line)
        df_cm = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test, predict))
        if cm.empty:
            cm = df_cm
        else:
            cm = cm.add(df_cm, fill_value=0)

    class_based_results = class_based_results / repetition
    results = pd.DataFrame(lines[1:], columns=lines[0])
    results.to_csv(output_csv, index=False)
    results = results.round(3)
    print(tabulate(results, headers=list(results.columns)))
    print()

    print(tabulate(class_based_results, headers=list(class_based_results.columns)))
    class_based_results.to_csv(output_csv.replace(".csv", "CB.csv"))
    if True:
        cm = cm // repetition
        cm.to_csv(output_csv.replace(".csv", "CM.csv"))



AD-S1-DI-S1
Epoch 1/10
1695/1695 [==============================] - 251s 144ms/step - loss: 2.9181 - accuracy: 0.0883 - val_loss: 3.3130 - val_accuracy: 0.0000e+00
Epoch 2/10
1695/1695 [==============================] - 245s 145ms/step - loss: 2.9162 - accuracy: 0.0897 - val_loss: 3.3373 - val_accuracy: 0.0000e+00
Epoch 3/10
1695/1695 [==============================] - 242s 143ms/step - loss: 2.9158 - accuracy: 0.0902 - val_loss: 3.3224 - val_accuracy: 0.0000e+00
Epoch 4/10
1695/1695 [==============================] - 244s 144ms/step - loss: 2.9161 - accuracy: 0.0885 - val_loss: 3.3003 - val_accuracy: 0.0000e+00
Epoch 5/10
1695/1695 [==============================] - 258s 152ms/step - loss: 2.9159 - accuracy: 0.0885 - val_loss: 3.3361 - val_accuracy: 0.0000e+00
Epoch 6/10
1695/1695 [==============================] - 245s 145ms/step - loss: 2.9159 - accuracy: 0.0901 - val_loss: 3.3108 - val_accuracy: 0.0000e+00
Epoch 7/10
1695/1695 [==============================] - 247s 146ms/step - l

Epoch 1/10
1716/1716 [==============================] - 340s 195ms/step - loss: 2.9456 - accuracy: 0.0785 - val_loss: 3.1129 - val_accuracy: 0.0000e+00
Epoch 2/10
1716/1716 [==============================] - 338s 197ms/step - loss: 2.9808 - accuracy: 0.0645 - val_loss: 3.0955 - val_accuracy: 0.0469
Epoch 3/10
1716/1716 [==============================] - 341s 199ms/step - loss: 2.9799 - accuracy: 0.0639 - val_loss: 3.0777 - val_accuracy: 0.0000e+00
Epoch 4/10
1716/1716 [==============================] - 341s 199ms/step - loss: 2.9787 - accuracy: 0.0675 - val_loss: 3.1124 - val_accuracy: 0.0000e+00
Epoch 5/10
1716/1716 [==============================] - 342s 199ms/step - loss: 2.9789 - accuracy: 0.0655 - val_loss: 3.1063 - val_accuracy: 0.0560
Epoch 6/10
1716/1716 [==============================] - 339s 198ms/step - loss: 2.9786 - accuracy: 0.0661 - val_loss: 3.1181 - val_accuracy: 0.0000e+00
Epoch 7/10
1716/1716 [==============================] - 341s 199ms/step - loss: 2.9787 - accurac

Epoch 1/10
1695/1695 [==============================] - 491s 285ms/step - loss: 2.6249 - accuracy: 0.1909 - val_loss: 3.1831 - val_accuracy: 0.0000e+00
Epoch 2/10
1695/1695 [==============================] - 463s 273ms/step - loss: 2.7948 - accuracy: 0.1217 - val_loss: 3.2168 - val_accuracy: 0.0000e+00
Epoch 3/10
1695/1695 [==============================] - 424s 250ms/step - loss: 2.8846 - accuracy: 0.0989 - val_loss: 3.3381 - val_accuracy: 0.0000e+00
Epoch 4/10
1695/1695 [==============================] - 411s 243ms/step - loss: 2.7577 - accuracy: 0.1422 - val_loss: 3.2615 - val_accuracy: 0.0000e+00
Epoch 5/10
1695/1695 [==============================] - 415s 245ms/step - loss: 2.7419 - accuracy: 0.1430 - val_loss: 3.3451 - val_accuracy: 0.0000e+00
Epoch 6/10
1695/1695 [==============================] - 465s 274ms/step - loss: 2.6445 - accuracy: 0.1686 - val_loss: 2.9936 - val_accuracy: 0.0977
Epoch 7/10
1695/1695 [==============================] - 479s 283ms/step - loss: 2.5957 - acc

Epoch 1/10
1719/1719 [==============================] - 481s 277ms/step - loss: 1.7727 - accuracy: 0.4468 - val_loss: 1.8884 - val_accuracy: 0.3424
Epoch 2/10
1719/1719 [==============================] - 507s 295ms/step - loss: 1.6617 - accuracy: 0.4835 - val_loss: 2.1380 - val_accuracy: 0.3439
Epoch 3/10
1719/1719 [==============================] - 640s 372ms/step - loss: 1.2600 - accuracy: 0.6000 - val_loss: 1.4968 - val_accuracy: 0.5346
Epoch 4/10
1719/1719 [==============================] - 720s 419ms/step - loss: 0.9708 - accuracy: 0.6888 - val_loss: 1.4945 - val_accuracy: 0.5542
Epoch 5/10
1719/1719 [==============================] - 722s 420ms/step - loss: 0.9198 - accuracy: 0.7032 - val_loss: 1.5106 - val_accuracy: 0.5326
Epoch 6/10
1719/1719 [==============================] - 721s 420ms/step - loss: 0.8731 - accuracy: 0.7181 - val_loss: 1.4713 - val_accuracy: 0.5255
Epoch 7/10
1719/1719 [==============================] - 726s 423ms/step - loss: 0.8640 - accuracy: 0.7215 - val_

KeyboardInterrupt: 

In [ ]:
file_list={#"DD11":['./small/AD-S1.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
#"DD11T":['./small/DI-S1.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
#"DD12":['./small/AD-S1.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
"DD12T":['./small/DI-S2.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
"DD22":['./small/AD-S2.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
"DD22T":['./small/DI-S2.csv','./small/AD-S2.csv','./small/AD-S1.csv'],
"DD21":['./small/AD-S2.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
"DD21T":['./small/DI-S1.csv','./small/AD-S2.csv','./small/AD-S1.csv']}
#file_list={"DD11":['./small/AD-S1.csv','./small/DI-S1.csv','./small/DI-S2.csv']}

In [20]:
import sklearn
import keras
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers 
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing



say = 0
dataset = "./results/other/1/"
folder(dataset)

for ii in file_list:
    print()
    say += 1
    train = file_list[ii][0]
    test = file_list[ii][1]
    mixed = False
    step = 1
    sayac = 1
    output_csv = f"{dataset}/LSTM@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}.csv"
    print(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}")
    target_names = target_name(test)

    class_based_results = pd.DataFrame()
    cm = pd.DataFrame()
    cv = 0
    repetition = 1
    lines = [["Dataset", "T", "CV", "ML", "Acc", "b_Acc", "Prec", "Rec", "F1", "kap", "ROC", "tra-T", "test-T", "Al-Time"]]

    for i in range(repetition):
        # Veri yükleme ve hazırlama
        df_train = pd.read_csv(file_list[ii][0], usecols=feature) 
        X_train = df_train.iloc[:, :-1]
        df_train['Label'] = df_train['Label'].astype('category')
        y_train = df_train['Label'].cat.codes  
        
        df_test = pd.read_csv(file_list[ii][1], usecols=feature) 
        X_test = df_test.iloc[:, :-1]
        df_test['Label'] = df_test['Label'].astype('category')
        y_test = df_test['Label'].cat.codes  
        
        min_max_scaler = preprocessing.MinMaxScaler()
        X_train = min_max_scaler.fit_transform(X_train)
        X_test = min_max_scaler.transform(X_test)
        
        # Veriyi LSTM modeline uygun hale getirme
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
        
        # Modelin oluşturulması
        model = keras.Sequential()
        model.add(layers.LSTM(units=32,
                              return_sequences=True,
                              input_shape=(X_train.shape[1], X_train.shape[2])))
        model.add(layers.Dropout(rate=0.1))
        
        model.add(layers.LSTM(units=64, return_sequences=True))
        model.add(layers.Dropout(rate=0.4))
        
        model.add(layers.LSTM(units=96, return_sequences=True))
        model.add(layers.Dropout(rate=0.3))
        
        model.add(layers.LSTM(units=96))
        model.add(layers.Dropout(rate=0.0))
        
        model.add(layers.Dense(21, activation='softmax'))  # 21 sınıf için
        
        model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01),
                      loss='sparse_categorical_crossentropy',
                      metrics=["accuracy"])
        
        second = time.time()
        model.fit(X_train, y_train, epochs=10,validation_split=0.3, verbose=1)
        results_y = []
        cv += 1
        results_y.append(y_test)
        train_time = (float((time.time() - second)))
        second = time.time()

        predict = model.predict(X_test)
        predict = np.argmax(predict, axis=1)
        test_time = (float((time.time() - second)))

        altime = 0
        dname = f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}"
        line, class_based_results = score(altime, train_time, test_time, predict, y_test, class_based_results, i, cv, dname, "LSTM")

        lines.append(line)
        df_cm = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test, predict))
        if cm.empty:
            cm = df_cm
        else:
            cm = cm.add(df_cm, fill_value=0)

    class_based_results = class_based_results / repetition
    results = pd.DataFrame(lines[1:], columns=lines[0])
    results.to_csv(output_csv, index=False)
    results = results.round(3)
    print(tabulate(results, headers=list(results.columns)))
    print()

    print(tabulate(class_based_results, headers=list(class_based_results.columns)))
    class_based_results.to_csv(output_csv.replace(".csv", "CB.csv"))
    if True:
        cm = cm // repetition
        cm.to_csv(output_csv.replace(".csv", "CM.csv"))



AD-S1-DI-S1
Epoch 1/10
1695/1695 [==============================] - 739s 411ms/step - loss: 2.4464 - accuracy: 0.2424 - val_loss: 2.2315 - val_accuracy: 0.3735
Epoch 2/10
1695/1695 [==============================] - 684s 404ms/step - loss: 2.3030 - accuracy: 0.2889 - val_loss: 2.9152 - val_accuracy: 0.0916
Epoch 3/10
1695/1695 [==============================] - 668s 394ms/step - loss: 2.0065 - accuracy: 0.3528 - val_loss: 1.7405 - val_accuracy: 0.4470
Epoch 4/10
1695/1695 [==============================] - 670s 395ms/step - loss: 1.6754 - accuracy: 0.4475 - val_loss: 1.6730 - val_accuracy: 0.4367
Epoch 5/10
1695/1695 [==============================] - 688s 406ms/step - loss: 2.2403 - accuracy: 0.3015 - val_loss: 2.4916 - val_accuracy: 0.2365
Epoch 6/10
1695/1695 [==============================] - 678s 400ms/step - loss: 1.8544 - accuracy: 0.3806 - val_loss: 1.8583 - val_accuracy: 0.3623
Epoch 7/10
1695/1695 [==============================] - 672s 397ms/step - loss: 1.6712 - accuracy: 

1695/1695 [==============================] - 538s 318ms/step - loss: 1.1228 - accuracy: 0.6330 - val_loss: 0.9696 - val_accuracy: 0.6612
Epoch 6/10
1695/1695 [==============================] - 525s 310ms/step - loss: 0.9167 - accuracy: 0.6954 - val_loss: 1.1258 - val_accuracy: 0.6152
Epoch 7/10
1695/1695 [==============================] - 540s 319ms/step - loss: 0.8264 - accuracy: 0.7156 - val_loss: 0.9893 - val_accuracy: 0.6047
Epoch 8/10
1695/1695 [==============================] - 541s 319ms/step - loss: 0.6275 - accuracy: 0.7783 - val_loss: 0.9091 - val_accuracy: 0.6477
Epoch 9/10
1695/1695 [==============================] - 523s 309ms/step - loss: 0.6956 - accuracy: 0.7593 - val_loss: 0.7291 - val_accuracy: 0.7369
Epoch 10/10
1695/1695 [==============================] - 525s 310ms/step - loss: 1.0894 - accuracy: 0.6583 - val_loss: 3.3711 - val_accuracy: 0.0051
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ---

1633/1633 [==============================] - 571s 350ms/step - loss: 2.6475 - accuracy: 0.1694 - val_loss: 2.9311 - val_accuracy: 0.0499
Epoch 10/10
1633/1633 [==============================] - 563s 345ms/step - loss: 2.6441 - accuracy: 0.1710 - val_loss: 2.9272 - val_accuracy: 0.0499
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S2-DI-S2    0     1  LSTM  0.149    0.126   0.063  0.126  0.061  0.096      0  5625.35   280.667          0

                                  precision    recall    f1-score       support
------------------------------  -----------  --------  ----------  ------------
Amazon Echo                       0          0           0          4592
Awair air quality monitor         0          0           0          3671
Belkin Wemo switch                0          0           0    

Epoch 1/10
1716/1716 [==============================] - 613s 354ms/step - loss: 2.1200 - accuracy: 0.3337 - val_loss: 1.8898 - val_accuracy: 0.3822
Epoch 2/10
1716/1716 [==============================] - 596s 348ms/step - loss: 1.1239 - accuracy: 0.6384 - val_loss: 1.4536 - val_accuracy: 0.6011
Epoch 3/10
1716/1716 [==============================] - 591s 344ms/step - loss: 0.9241 - accuracy: 0.7071 - val_loss: 1.1067 - val_accuracy: 0.6937
Epoch 4/10
1716/1716 [==============================] - 590s 344ms/step - loss: 0.8743 - accuracy: 0.7282 - val_loss: 2.4332 - val_accuracy: 0.2688
Epoch 5/10
1716/1716 [==============================] - 603s 352ms/step - loss: 1.2960 - accuracy: 0.5606 - val_loss: 1.8881 - val_accuracy: 0.4176
Epoch 6/10
1716/1716 [==============================] - 607s 353ms/step - loss: 1.3734 - accuracy: 0.5460 - val_loss: 1.9971 - val_accuracy: 0.4041
Epoch 7/10
1716/1716 [==============================] - 653s 381ms/step - loss: 1.6855 - accuracy: 0.4587 - val_

In [21]:
!shutdown /s /t 360